# 환경 설정 & KoBERT 설치
- https://github.com/ChangZero/koBERT-finetuning-demo/blob/main/kobert_colab.ipynb

In [1]:
# # install and restart
# !pip install mxnet
# !pip install gluonnlp==0.8.0
# !pip install tqdm pandas
# !pip install sentencepiece
# !pip install transformers
# !pip install torch>=1.8.1
# !pip install transformers

# # !pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

In [35]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.model_selection import train_test_split
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
from tqdm.notebook import tqdm
import datetime

In [36]:
# torch GPU 설정
device_type = 'cuda' if torch.cuda.is_available() else 'cpu'
device = torch.device(device_type)
device

device(type='cuda')

# git clone data

In [3]:
!git clone https://github.com/huijunam/CJ_AI_hackathon.git

Cloning into 'CJ_AI_hackathon'...
remote: Enumerating objects: 85, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 85 (delta 7), reused 0 (delta 0), pack-reused 69
Receiving objects: 100% (85/85), 4.15 MiB | 4.39 MiB/s, done.
Resolving deltas: 100% (35/35), done.


In [4]:
import pandas as pd

chatbot_data_1 = pd.read_excel('/content/CJ_AI_hackathon/data/8f3202096f1387c5.xlsx')
chatbot_data_2 = pd.read_excel('/content/CJ_AI_hackathon/data/ad8cb80ea8623bbe.xlsx')

In [5]:
chatbot_data_1.sample(n =10)

,Sentence,Emotion,Unnamed: 2,Unnamed: 3,Unnamed: 4,공포,5468
3479,이름도 무섭다....,공포,NaN,NaN,NaN,NaN,NaN
25583,그냥 번호를 없애자,중립,NaN,NaN,NaN,NaN,NaN
34821,대통령은 탇핵의 대상이 맞읍니다.,혐오,NaN,NaN,NaN,NaN,NaN
37144,ㅋㅋㅋㅋ뭔 빈민촌도 아니고 파라솔 다다닥~~저짓하러 저까지 왜가는거냐 대체 ㅋㅋ,혐오,NaN,NaN,NaN,NaN,NaN
32195,앙이들 화이팅!,행복,NaN,NaN,NaN,NaN,NaN
30166,문의사항이나 제보는 아래로 연락주시면 감사하겠습니다.,행복,NaN,NaN,NaN,NaN,NaN
31217,사랑해... 미안해...고마워.... 보고싶다 ㅠㅠ,행복,NaN,NaN,NaN,NaN,NaN
15800,인민재판도이러지는안하지,분노,NaN,NaN,NaN,NaN,NaN
23798,음~~알파고를 이길사람은 나뿐이군...ㅋ,중립,NaN,NaN,NaN,NaN,NaN
32847,시위 참여한 사람인데 진짜 우리 국민들이 한마음이라는 생각이 들었어요,행복,NaN,NaN,NaN,NaN,NaN


In [6]:
chatbot_data_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38594 entries, 0 to 38593
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Sentence    38594 non-null  object 
 1   Emotion     38594 non-null  object 
 2   Unnamed: 2  0 non-null      float64
 3   Unnamed: 3  0 non-null      float64
 4   Unnamed: 4  0 non-null      float64
 5   공포          7 non-null      object 
 6   5468        7 non-null      float64
dtypes: float64(4), object(3)
memory usage: 2.1+ MB


In [7]:
chatbot_data_1.describe()

,Unnamed: 2,Unnamed: 3,Unnamed: 4,5468
count,0.0,0.0,0.0,7.000000
mean,NaN,NaN,NaN,10245.714286
std,NaN,NaN,NaN,12506.947639
min,NaN,NaN,NaN,4830.000000
25%,NaN,NaN,NaN,5348.000000
50%,NaN,NaN,NaN,5665.000000
75%,NaN,NaN,NaN,5967.500000
max,NaN,NaN,NaN,38594.000000


In [8]:
chatbot_data_1.count()

Sentence      38594
Emotion       38594
Unnamed: 2        0
Unnamed: 3        0
Unnamed: 4        0
공포                7
5468              7
dtype: int64

In [9]:
# 필요없는 열 삭제
chatbot_data_1= chatbot_data_1.iloc[:,:2]
chatbot_data_1

,Sentence,Emotion
0,언니 동생으로 부르는게 맞는 일인가요..??,공포
1,그냥 내 느낌일뿐겠지?,공포
2,아직너무초기라서 그런거죠?,공포
3,유치원버스 사고 낫다던데,공포
4,근데 원래이런거맞나요,공포
...,...,...
38589,솔직히 예보 제대로 못하는 데 세금이라도 아끼게 그냥 폐지해라..,혐오
38590,재미가 없으니 망하지,혐오
38591,공장 도시락 비우생적임 아르바이트했는데 화장실가성 손도 않씯고 재료 담고 바닥 떨어...,혐오
38592,코딱지 만한 나라에서 지들끼리 피터지게 싸우는 센징 클래스 ㅉㅉㅉ,혐오


In [10]:
chatbot_data_1.isnull().sum() # 결측치 확인

Sentence    0
Emotion     0
dtype: int64

In [11]:
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

chatbot_data_1['Emotion'].value_counts()

행복    6037
놀람    5898
분노    5665
공포    5468
혐오    5429
슬픔    5267
중립    4830
Name: Emotion, dtype: int64

In [12]:
chatbot_data_2.sample(n =10)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,행복,중립,슬픔,공포,혐오,분노,놀람
16195,NaN,"세미나하다가 논쟁이 붙었는데, 혼자 흥분해서 그래요.",중립,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27810,NaN,내 동생 어제 결국 집에 안들어오고 병원갔지?,중립,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45308,NaN,응? 있어....친구.....,중립,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
41915,NaN,"그렇다고 그거 몇분 와서 얼굴 들이민거 가지구, 이십만원이라니? 말도 안돼지 그건,...",분노,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29673,NaN,창문은요?,중립,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11120,NaN,우린 같이 가기엔 너무 멀어진 것 같다.,중립,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54651,NaN,"아니, 엄마가 만들어 줄 때는 평범하게 먹어. 오늘은 장 보는 걸 깜빡하는 바람에 ...",중립,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31045,NaN,뭐야 갑자기 왜 이래?,중립,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22538,NaN,타임머신 타고 왔나보지. 머나먼 과거에서...,중립,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
34501,NaN,선생님.. 잊어버리셨죠...?,중립,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
chatbot_data_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55629 entries, 0 to 55628
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  10001 non-null  object 
 1   Unnamed: 1  55628 non-null  object 
 2   Unnamed: 2  55622 non-null  object 
 3   Unnamed: 3  0 non-null      float64
 4   Unnamed: 4  0 non-null      float64
 5   행복          1 non-null      float64
 6   중립          0 non-null      float64
 7   슬픔          0 non-null      float64
 8   공포          0 non-null      float64
 9   혐오          0 non-null      float64
 10  분노          0 non-null      float64
 11  놀람          0 non-null      float64
dtypes: float64(9), object(3)
memory usage: 5.1+ MB


In [14]:
chatbot_data_2.isnull().sum() # 결측치 확인

Unnamed: 0    45628
Unnamed: 1        1
Unnamed: 2        7
Unnamed: 3    55629
Unnamed: 4    55629
행복            55628
중립            55629
슬픔            55629
공포            55629
혐오            55629
분노            55629
놀람            55629
dtype: int64

In [15]:
chatbot_data_2 = chatbot_data_2.iloc[1:, :3] # 필요없는 열 삭제
chatbot_data_2

,Unnamed: 0,Unnamed: 1,Unnamed: 2
1,S,아 진짜! 사무실에서 피지 말라니깐! 간접흡연이 얼마나 안좋은데!,분노
2,NaN,그럼 직접흡연하는 난 얼마나 안좋겠니? 안그래? 보면 꼭... 지 생각만 하고.,혐오
3,NaN,손님 왔어요.,중립
4,NaN,손님? 누구?,중립
5,NaN,몰라요. 팀장님 친구래요.,중립
...,...,...,...
55624,S,얘긴 다 끝났냐? 원예부,중립
55625,NaN,"예. 그거 때문에, 부탁이 있......는......데요.",중립
55626,NaN,여자 숨겨달라는거면 사절이다.,중립
55627,NaN,아무래도 안되나요?,중립


In [16]:
chatbot_data_2[chatbot_data_2['Unnamed: 0'] =='S']

,Unnamed: 0,Unnamed: 1,Unnamed: 2
1,S,아 진짜! 사무실에서 피지 말라니깐! 간접흡연이 얼마나 안좋은데!,분노
8,S,그래서... 무슨 일 해?,중립
16,S,언제부터 시작할 수 있어?,중립
30,S,안 내켜.,혐오
42,S,"집 짓는 이유가 그렇게 중요해? 왜, 내가 사기라도 칠까봐?",혐오
...,...,...,...
55604,S,자네는 대체 뭘 하러 왔나! 젖은 생쥐 꼴이 된 나를 보면서 비웃으러 왔나?,분노
55609,S,조사결과에 따르면 의뢰인의 아버지는 횡령같은 걸 하지 않았다고 하는군.,중립
55614,S,그게 무슨 소리야?,중립
55619,S,기억해 두게. 이 사건에서 사실은 필요치 않아.,중립


In [17]:
chatbot_data_2 = chatbot_data_2.iloc[:,1:] # Unnamed: 0 열 삭제
chatbot_data_2

,Unnamed: 1,Unnamed: 2
1,아 진짜! 사무실에서 피지 말라니깐! 간접흡연이 얼마나 안좋은데!,분노
2,그럼 직접흡연하는 난 얼마나 안좋겠니? 안그래? 보면 꼭... 지 생각만 하고.,혐오
3,손님 왔어요.,중립
4,손님? 누구?,중립
5,몰라요. 팀장님 친구래요.,중립
...,...,...
55624,얘긴 다 끝났냐? 원예부,중립
55625,"예. 그거 때문에, 부탁이 있......는......데요.",중립
55626,여자 숨겨달라는거면 사절이다.,중립
55627,아무래도 안되나요?,중립


In [18]:
chatbot_data_2['Unnamed: 2'].value_counts()

중립     43786
놀람      4866
분노      3628
슬픔      1972
행복      1030
혐오       220
공포        98
ㅍ         12
분          4
ㅈ중립        1
분ㄴ         1
중림         1
ㄴ중립        1
줄          1
Name: Unnamed: 2, dtype: int64

In [19]:
chatbot_data_2[chatbot_data_2['Unnamed: 2'] == 'ㅍ'] # 확인

,Unnamed: 1,Unnamed: 2
25037,인터넷에다 팔아야 되나.. 근데 너 이제 맘 잡았나보다?,ㅍ
27177,일부러 그럴 리가 있어요?,ㅍ
27948,그럼요?,ㅍ
28090,샤프심이 없어서..가지러 왔다가 누나 보이길래..,ㅍ
28455,아~ 자기소개서 쓰기 힘들어 죽겠어요~,ㅍ
28482,원래 남자들..다른 남자랑 친하게 지내면 그러니까..,ㅍ
36684,"그럼, 임마! 내가 누구냐? 한번 실수는 병가지상사다~이러시면서 나의 인간성과 품위...",ㅍ
37261,"처리 할거야. 처리해야지, 그게 돈이 얼만데?",ㅍ
37428,"와, 어디서 꽁돈이라도 생겼나?",ㅍ
37864,"엄마, 엄만 결혼이 뭐 그리 좋아?",ㅍ


In [20]:
# 90 이하인 카테고리 추출
value_counts = chatbot_data_2['Unnamed: 2'].value_counts()
categories_to_remove = value_counts[value_counts <= 90].index

# 해당 카테고리를 포함하지 않는 행만 선택
chatbot_data_2 = chatbot_data_2[~chatbot_data_2['Unnamed: 2'].isin(categories_to_remove)]

In [21]:
chatbot_data_2['Unnamed: 2'].value_counts()

중립    43786
놀람     4866
분노     3628
슬픔     1972
행복     1030
혐오      220
공포       98
Name: Unnamed: 2, dtype: int64

In [22]:
chatbot_data_2.columns =['Sentence', 'Emotion']

In [23]:
chatbot_data_2

,Sentence,Emotion
1,아 진짜! 사무실에서 피지 말라니깐! 간접흡연이 얼마나 안좋은데!,분노
2,그럼 직접흡연하는 난 얼마나 안좋겠니? 안그래? 보면 꼭... 지 생각만 하고.,혐오
3,손님 왔어요.,중립
4,손님? 누구?,중립
5,몰라요. 팀장님 친구래요.,중립
...,...,...
55624,얘긴 다 끝났냐? 원예부,중립
55625,"예. 그거 때문에, 부탁이 있......는......데요.",중립
55626,여자 숨겨달라는거면 사절이다.,중립
55627,아무래도 안되나요?,중립


In [24]:
chatbot_data_1['Emotion'].unique()

array(['공포', '놀람', '분노', '슬픔', '중립', '행복', '혐오'], dtype=object)

In [25]:
chatbot_data_2['Emotion'].unique()
chatbot_data_2 = chatbot_data_2.dropna(how='any')# Null 값 제거

In [26]:
chatbot_data_2['Emotion'].value_counts()

중립    43786
놀람     4866
분노     3628
슬픔     1972
행복     1030
혐오      220
공포       98
Name: Emotion, dtype: int64

In [27]:
chatbot_data_2[chatbot_data_2['Emotion'] == '중립'] # 확인

,Sentence,Emotion
3,손님 왔어요.,중립
4,손님? 누구?,중립
5,몰라요. 팀장님 친구래요.,중립
6,내 친구? 친구 누구?,중립
8,그래서... 무슨 일 해?,중립
...,...,...
55624,얘긴 다 끝났냐? 원예부,중립
55625,"예. 그거 때문에, 부탁이 있......는......데요.",중립
55626,여자 숨겨달라는거면 사절이다.,중립
55627,아무래도 안되나요?,중립


In [28]:
len(chatbot_data_1), len(chatbot_data_2)

(38594, 55600)

In [29]:
df_data = pd.concat([chatbot_data_1,chatbot_data_2], ignore_index=True)
df_data # 최종 데이터

,Sentence,Emotion
0,언니 동생으로 부르는게 맞는 일인가요..??,공포
1,그냥 내 느낌일뿐겠지?,공포
2,아직너무초기라서 그런거죠?,공포
3,유치원버스 사고 낫다던데,공포
4,근데 원래이런거맞나요,공포
...,...,...
94189,얘긴 다 끝났냐? 원예부,중립
94190,"예. 그거 때문에, 부탁이 있......는......데요.",중립
94191,여자 숨겨달라는거면 사절이다.,중립
94192,아무래도 안되나요?,중립


In [30]:
df_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94194 entries, 0 to 94193
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Sentence  94194 non-null  object
 1   Emotion   94194 non-null  object
dtypes: object(2)
memory usage: 1.4+ MB


# 데이터셋 전처리

In [31]:
target_classes ={
    '공포': 0,
    '놀람': 1,
    '분노': 2,
    '슬픔': 3,
    '중립': 4,
    '행복': 5,
    '혐오': 6
  }

In [32]:
# 레이블링
for cls in target_classes:
  df_data.loc[(df_data['Emotion'] == cls), 'Emotion'] = target_classes[cls]

In [33]:
data_list =[]
# ['sentence', 'class'] 형태로 변환
for q, label in zip(df_data['Sentence'], df_data['Emotion']):
  data =[]
  data.append(q)
  data.append(str(label))

  data_list.append(data)

In [34]:
print(data_list[6000])

['기술적으로도 아직도 해체해서 다시 완벽히 돌려놓는게 어려운데 해체를한다고?', '1']


In [176]:
# BERTSentenceTransform 수정
class BERTSentenceTransform:
    r"""BERT style data transformation.

    Parameters
    ----------
    tokenizer : BERTTokenizer.
        Tokenizer for the sentences.
    max_seq_length : int.
        Maximum sequence length of the sentences.
    pad : bool, default True
        Whether to pad the sentences to maximum length.
    pair : bool, default True
        Whether to transform sentences or sentence pairs.
    """

    def __init__(self, tokenizer, max_seq_length, vocab, pad=True, pair=True):
        self._tokenizer = tokenizer
        self._max_seq_length = max_seq_length
        self._pad = pad
        self._pair = pair
        self._vocab = vocab

    def __call__(self, line):
        """Perform transformation for sequence pairs or single sequences.

        The transformation is processed in the following steps:
        - tokenize the input sequences
        - insert [CLS], [SEP] as necessary
        - generate type ids to indicate whether a token belongs to the first
        sequence or the second sequence.
        - generate valid length

        For sequence pairs, the input is a tuple of 2 strings:
        text_a, text_b.

        Inputs:
            text_a: 'is this jacksonville ?'
            text_b: 'no it is not'
        Tokenization:
            text_a: 'is this jack ##son ##ville ?'
            text_b: 'no it is not .'
        Processed:
            tokens: '[CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]'
            type_ids: 0     0  0    0    0     0       0 0     1  1  1  1   1 1
            valid_length: 14

        For single sequences, the input is a tuple of single string:
        text_a.

        Inputs:
            text_a: 'the dog is hairy .'
        Tokenization:
            text_a: 'the dog is hairy .'
        Processed:
            text_a: '[CLS] the dog is hairy . [SEP]'
            type_ids: 0     0   0   0  0     0 0
            valid_length: 7

        Parameters
        ----------
        line: tuple of str
            Input strings. For sequence pairs, the input is a tuple of 2 strings:
            (text_a, text_b). For single sequences, the input is a tuple of single
            string: (text_a,).

        Returns
        -------
        np.array: input token ids in 'int32', shape (batch_size, seq_length)
        np.array: valid length in 'int32', shape (batch_size,)
        np.array: input token type ids in 'int32', shape (batch_size, seq_length)

        """

        # convert to unicode
        text_a = line[0]
        if self._pair:
            assert len(line) == 2
            text_b = line[1]

        tokens_a = self._tokenizer.tokenize(text_a) # 수정한 부분
        tokens_b = None

        if self._pair:
            tokens_b = self._tokenizer(text_b)

        if tokens_b:
            # Modifies `tokens_a` and `tokens_b` in place so that the total
            # length is less than the specified length.
            # Account for [CLS], [SEP], [SEP] with "- 3"
            self._truncate_seq_pair(tokens_a, tokens_b,
                                    self._max_seq_length - 3)
        else:
            # Account for [CLS] and [SEP] with "- 2"
            if len(tokens_a) > self._max_seq_length - 2:
                tokens_a = tokens_a[0:(self._max_seq_length - 2)]

        # The embedding vectors for `type=0` and `type=1` were learned during
        # pre-training and are added to the wordpiece embedding vector
        # (and position vector). This is not *strictly* necessary since
        # the [SEP] token unambiguously separates the sequences, but it makes
        # it easier for the model to learn the concept of sequences.

        # For classification tasks, the first vector (corresponding to [CLS]) is
        # used as as the "sentence vector". Note that this only makes sense because
        # the entire model is fine-tuned.
        #vocab = self._tokenizer.vocab
        vocab = self._vocab
        tokens = []
        tokens.append(vocab.cls_token)
        tokens.extend(tokens_a)
        tokens.append(vocab.sep_token)
        segment_ids = [0] * len(tokens)

        if tokens_b:
            tokens.extend(tokens_b)
            tokens.append(vocab.sep_token)
            segment_ids.extend([1] * (len(tokens) - len(segment_ids)))

        input_ids = self._tokenizer.convert_tokens_to_ids(tokens)

        # The valid length of sentences. Only real  tokens are attended to.
        valid_length = len(input_ids)

        if self._pad:
            # Zero-pad up to the sequence length.
            padding_length = self._max_seq_length - valid_length
            # use padding tokens for the rest
            input_ids.extend([vocab[vocab.padding_token]] * padding_length)
            segment_ids.extend([0] * padding_length)

        return np.array(input_ids, dtype='int32'), np.array(valid_length, dtype='int32'),\
            np.array(segment_ids, dtype='int32')

In [177]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len,
                 pad, pair):
        transform = BERTSentenceTransform(bert_tokenizer, max_seq_length=max_len,vocab=vocab, pad=pad, pair=pair)
        #transform = nlp.data.BERTSentenceTransform(
        #    tokenizer, max_seq_length=max_len, pad=pad, pair=pair)
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

# wandb 설정

In [ ]:
# wandb 설치
!pip install wandb

In [180]:
!wandb login 09b76e2d7910cb85b58be240c8cf088ff601d126

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [181]:
import wandb
# project: 폴더 이름, entity: team_5g (팀), name: 기록명
test_time = datetime.datetime.now()
extra ='' # 원본 데이터 일 경우
# extra ='drop_data_' # drop data 일 경우
wandb.init(project="koBERT",name=f"epoch_{num_epochs}_{extra}batch_size_{batch_size}_learning_rate_{learning_rate}/{test_time}",notes=f"koBERT-epoch_{num_epochs}-batch_size_{batch_size}-random_seed{random_seed}-learning_rate_{learning_rate}", entity='team_5g')

train_acc,▁▅█
train_loss,█▃▁
val_acc,▃▄▃▄▅▄▅▅▅▅▅▆▆▁▄▆▇██▇▇▇▇▇█▇▇█▆███▇█▇▇▇███
val_loss,▄▄▆▆█▅▁▇▃▄▂▄▅█▄▁▂▄▆▅▃▂▃▅▅▅▄▂▇▅▃▅▆▇▃▅▃▅▄▄
train_acc,0.77546
val_acc,0.69026


In [182]:
# config
wandb.config ={
  "learning_rate": learning_rate,
  "epochs": num_epochs,
  "batch_size": batch_size,
  "seed": random_seed
}

In [183]:
# loss 추적
wandb.define_metric('train_loss', summary='min')
wandb.define_metric('val_loss', summary='min')
# accuracy score 추적
wandb.define_metric('train_accuracy', summary='max')
wandb.define_metric('val_accuracy', summary='max')

In [184]:
from kobert_tokenizer import KoBERTTokenizer
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [186]:
# kobert 공식 git에 있는 get_kobert_model 선언
def get_kobert_model(model_path, vocab_file, ctx=device):
    bertmodel = BertModel.from_pretrained(model_path)
    device = torch.device(ctx)
    bertmodel.to(device)
    bertmodel.eval()
    vocab_b_obj = nlp.vocab.BERTVocab.from_sentencepiece(vocab_file,
                                                         padding_token='[PAD]')
    return bertmodel, vocab_b_obj

In [187]:
import gluonnlp as nlp
from transformers import BertModel
bertmodel, vocab = get_kobert_model('skt/kobert-base-v1',tokenizer.vocab_file)
# tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower = False)

In [44]:
train_list, test_list = train_test_split(data_list, test_size = 0.2, shuffle = True, random_state = random_seed)

In [189]:
data_train = BERTDataset(train_list, 0, 1, tokenizer, vocab, max_len, True, False)
data_test = BERTDataset(test_list, 0, 1, tokenizer, vocab, max_len, True, False)

In [115]:
data_train[0] # 토큰화와 패딩 확인

(array([   2, 1563, 6964, 6604, 6857,    5,    3,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1], dtype=int32),
 array(7, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [190]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size = batch_size, num_workers = 2)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size = batch_size, num_workers = 2)

In [191]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes = len(target_classes),   # 클래스 현재 7개
                 dr_rate = None,
                 params = None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p = dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device),return_dict = False)
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [192]:
model = BERTClassifier(bertmodel, dr_rate = 0.5).to(device)

In [76]:
# no_decay = ['bias', 'LayerNorm.weight']
# optimizer_grouped_parameters = [
#     {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
#     {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
# ]

# optimizer = AdamW(optimizer_grouped_parameters, lr = learning_rate)
# loss_fn = nn.CrossEntropyLoss() # 다중분류를 위한 loss function

# # t_total = len(train_dataloader) * num_epochs
# warmup_step = int(t_total * warmup_ratio)

# scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps = warmup_step, num_training_steps = t_total)


NameError: ignored

In [194]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [58]:
# 하이퍼 파라미터 설정
max_length = 100
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5
random_seed = 42

# KoBERT 모델 학습

In [195]:
import time

# train_history = []
# test_history = []
# loss_history = []
y_preds =[]
label_list =[]

results ={
      "train_loss": [],
      "train_acc": [],
      "val_loss": [],
      "val_acc": []
  }

# watch model
wandb.watch(model)
for e in range(num_epochs):
  train_acc = 0.0
  test_acc = 0.0

  model.train() #train
  start_time = time.time() # time

  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(train_dataloader)):
    optimizer.zero_grad()
    token_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)
    valid_length= valid_length
    label = label.long().to(device)

    y_pred = model(token_ids, valid_length, segment_ids)

    # print(label.shape, out.shape)
    train_loss = loss_fn(y_pred, label)
    train_loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
    optimizer.step()
    scheduler.step()  # Update learning rate schedule
    train_acc += calc_accuracy(y_pred, label)

    if batch_id % log_interval == 0:
      print("epoch {} batch id {} loss {:.4f} train acc {:.4f}".format(e+1, batch_id+1, train_loss.data.cpu().numpy(), train_acc / (batch_id+1)))
      # train_history.append(train_acc / (batch_id+1))
      # loss_history.append(loss.data.cpu().numpy())
      results["train_loss"].append(train_loss.data.cpu().numpy())
      results["train_acc"].append(train_acc / (batch_id+1))

    # wandb log
    wandb.log({"train_acc": train_acc / (batch_id+1)}, commit=False)
    wandb.log({"train_loss": train_loss.data.cpu().numpy()}, commit=False)

  print("epoch {} train acc {:.4f}".format(e+1, train_acc / (batch_id+1)))

  model.eval() # inference
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(test_dataloader)):
    token_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)
    valid_length = valid_length
    label = label.long().to(device)
    test_pred = model(token_ids, valid_length, segment_ids)

    y_preds.extend(test_pred.cpu().detach().numpy()) # for confusion matrix
    label_list.extend(label.cpu().numpy()) # for confusion matrix true label

    test_loss = loss_fn(test_pred, label)
    test_loss.backward()

    test_acc += calc_accuracy(test_pred, label)
    # wandb log
    wandb.log({"val_acc": test_acc/(batch_id+1)}, commit=False)
    wandb.log({"val_loss": test_loss.data.cpu().numpy()})

    results["val_loss"].append(test_loss.data.cpu().numpy())
    results["val_acc"].append(test_acc/(batch_id+1))

  end_time = time.time()
  print("epoch {} test acc {:.4f}".format(e+1, test_acc / (batch_id+1)))
  print("Time: {:.4f}sec".format((end_time - start_time)))

  # wandb log
  # wandb.log({'train_loss': results["train_loss"], 'val_loss': results["val_loss"], 'train_accuracy': results["train_acc"], 'val_accuracy': results["val_acc"]}, step=e)
  # test_history.append(test_acc / (batch_id+1))

label_list = np.array(label_list)
y_preds = np.array(y_preds)

  0%|          | 0/570 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 1.8044 train acc 0.1875
epoch 1 batch id 201 loss 1.2860 train acc 0.3267
epoch 1 batch id 401 loss 1.2849 train acc 0.4274
epoch 1 train acc 0.4696


  0%|          | 0/143 [00:00<?, ?it/s]

epoch 1 test acc 0.5936
Time: 651.1601sec


  0%|          | 0/570 [00:00<?, ?it/s]

...

epoch 4 test acc 0.6203
Time: 660.6919sec


  0%|          | 0/570 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.5726 train acc 0.7812
epoch 5 batch id 201 loss 0.3532 train acc 0.8193
epoch 5 batch id 401 loss 0.3953 train acc 0.8314
epoch 5 train acc 0.8361


  0%|          | 0/143 [00:00<?, ?it/s]

epoch 5 test acc 0.6213
Time: 661.8787sec


In [200]:
y_preds_label = [np.argmax(l) for l in y_preds]
len(y_preds_label), len(label_list)

(45580, 45580)

In [203]:
len(y_preds_label)

45580

In [123]:
# torch.save(model.state_dict(), "./base_model_koBERT_epoch3_.pt")

# KoELECTRA 모델
https://github.com/monologg/KoELECTRA

## Dataset

In [ ]:
# Load the pre-trained LLM model and tokenizer
model_name = "monologg/koelectra-small-v2-discriminator"
tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels =len(target_classes))
model.to(device)

In [92]:
# Define a custom dataset class
class custom_dataset(Dataset):
  def __init__(self, texts, labels, tokenizer, max_length):
    self.texts = texts
    self.labels = labels
    self.tokenizer = tokenizer
    self.max_length = max_length

  def __len__(self):
    return len(self.texts)

  def __getitem__(self, idx):
    text = self.texts[idx]
    label = self.labels[idx]
    encoding = self.tokenizer(text,padding='max_length', truncation=True, max_length=self.max_length, return_tensors='pt')

    return {
        'input_ids': encoding['input_ids'].squeeze(),
        'attention_mask': encoding['attention_mask'].squeeze(),
        'labels': torch.tensor(int(label), dtype=torch.float)
        # 'labels': torch.tensor(int(label), dtype=torch.long)
    }

In [93]:
# Define the custom collate function
def collate_fn(batch):
 input_ids = [item['input_ids'] for item in batch]
 attention_masks = [item['attention_mask'] for item in batch]
 labels = [item['labels'] for item in batch]

 # Pad or truncate sequences to the same length within each batch
 input_ids = torch.nn.utils.rnn.pad_sequence(input_ids, batch_first=True)
 attention_masks = torch.nn.utils.rnn.pad_sequence(attention_masks, batch_first=True)
 labels = torch.stack(labels)

 return {
 'input_ids': input_ids,
 'attention_mask': attention_masks,
 'labels': labels
 }

In [94]:
train_list, test_list = train_test_split(data_list, test_size = 0.2, shuffle = True, random_state = random_seed)
train_list = pd.DataFrame(train_list, columns=['Sentence', 'Emotion'])
test_list = pd.DataFrame(test_list, columns=['Sentence', 'Emotion'])

# Load your training data
X_train = train_list['Sentence']
Y_train = train_list['Emotion']

# Load your testing data
X_test = test_list['Sentence']
Y_test = test_list['Emotion']

In [95]:
# dataset
train_dataset = custom_dataset(X_train, Y_train, tokenizer, max_length)
test_dataset = custom_dataset(X_test, Y_test, tokenizer, max_length)

# dataloader
train_data_loader = DataLoader(train_dataset,batch_size=batch_size,shuffle=True,collate_fn=collate_fn)
test_data_loader = DataLoader(test_dataset,batch_size=batch_size,shuffle=True,collate_fn=collate_fn)

In [105]:
# text, attention_mask, y = train_dataset[0]
# attention_mask.shape

torch.Size([100])

# 학습

In [96]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
criterion = nn.CrossEntropyLoss()

In [101]:
for epoch in range(num_epochs):
  model.train()
  total_loss = 0

  # train
  for step, batch in enumerate(train_data_loader):
    optimizer.zero_grad()
    input_ids = batch['input_ids'].to(device)
    # input_ids = input_ids.to(device)
    attention_mask = batch['attention_mask'].to(device)
    # attention_mask = attention_mask.to(device)
    labels = batch['labels'].to(device)
    # labels = labels.to(device)
    outputs = model(input_ids=input_ids, attention_mask=attention_mask)
    logits = outputs.logits

    # Calculate loss
    loss = criterion(logits.squeeze(), labels)
    total_loss += loss.item()
    loss.backward()
    optimizer.step()

    if (step + 1) % 200 == 0:
      print(f"Epoch {epoch + 1}/{num_epochs} | Step {step + 1}/{len(train_data_loader)} | Loss: {loss.item():.4f}")

  avg_loss = total_loss / len(train_data_loader)
  print(f"Epoch {epoch + 1}/{num_epochs} | Loss: {avg_loss:.4f}")

  # eval
  val_total_loss = 0
  correct = 0
  total = 0

  with torch.no_grad():
    for step, batch in enumerate(test_data_loader):
      val_input_ids = batch['input_ids'].to(device)
      val_attentio_mask = batch['attention_mask'].to(device)
      val_labels = batch['labels'].to(device)

      val_outputs = model(input_ids=input_ids,attention_mask=attention_mask)
      val_logits = val_outputs.logits.squeeze()
      val_predictions = val_logits.detach().cpu().numpy()
      val_targets = val_labels.detach().cpu().numpy()

      # Calculate loss
      val_loss = criterion(val_logits, val_labels)
      val_total_loss +=val_loss.items()

      # Calculate accuracy
      val_preds = val_logits.argmax(dim=1)
      correct +=(val_preds == val_labels).sum().item()
      total +=val_labels.size(0)

  val_avg_loss = val_total_loss / len(test_data_loader)
  val_accuracy = correct / total
  print(f"Validation Loss: {val_avg_loss:.4f} - Validation Accuracy: {val_accuracy:.4f}")

Epoch 1/5 | Step 200/1178 | Loss: 0.8930
Epoch 1/5 | Step 400/1178 | Loss: 1.0246
Epoch 1/5 | Step 600/1178 | Loss: 0.9306
Epoch 1/5 | Step 800/1178 | Loss: 0.8281
Epoch 1/5 | Step 1000/1178 | Loss: 1.2174
Epoch 1/5 | Loss: 1.0446


ValueError: ignored

# 평가

In [ ]:
from sklearn.metrics import confusion_matrix
from mlxtend.plotting import plot_confusion_matrix
import seaborn as sns

cm = confusion_matrix(label_list, y_preds_label)
# Define class labels (assuming 7 classes in this example)
class_labels = list(dict(map(reversed, target_classes.items())).keys()) # true label

plt.figure(figsize=(8, 6))
fig, ax = plot_confusion_matrix(conf_mat=cm, colorbar=True,
show_absolute=False, show_normed=True, class_names=class_labels)
plt.show()

# Create a heatmap for the confusion matrix
# plt.figure(figsize=(8, 6))
# sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
# plt.xlabel('Predicted Labels')
# plt.ylabel('True Labels')
# plt.title('Confusion Matrix')
# plt.show()

In [125]:
# wandb_img = wandb.Image(cm, caption=f"basemodel_epoch_{num_epochs}_figure")

# wandb.log({"Confusion Matrix": wandb_img})

# 키워드 추출
- 명사, 동사, 형용사

In [ ]:
!pip install konlpy

In [61]:
from konlpy.tag import Okt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 60.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 47.0 MB/s eta 0:00:00


In [62]:
okt = Okt()
kor_sentence = '너무 배고파 ㅜㅜ'
print('OKT 품사 태깅 :', okt.pos(kor_sentence))

OKT 품사 태깅 : [('너무', 'Adverb'), ('배고파', 'Adjective'), ('ㅜㅜ', 'KoreanParticle')]


In [63]:
# 명사, 동사, 형용사 추출하기
keyword = [x for (x, y) in okt.pos(kor_sentence) if y in ['Verb', 'Noun', 'Adjective']]
print('\nOKT 명사, 동사, 형용사 추출 :', keyword)


OKT 명사, 동사, 형용사 추출 : ['배고파']


In [64]:
# okt.normalize(kor_sentence)

# 새로운 문장 테스트

In [65]:
# target_classes ={
#     '공포': 0,
#     '놀람': 1,
#     '분노': 2,
#     '슬픔': 3,
#     '중립': 4,
#     '행복': 5,
#     '혐오': 6
#   }

In [66]:
reverse =dict(map(reversed, target_classes.items()))
print(reverse)

{0: '공포', 1: '놀람', 2: '분노', 3: '슬픔', 4: '중립', 5: '행복', 6: '혐오'}


In [67]:
list(reverse.keys())

[0, 1, 2, 3, 4, 5, 6]

In [68]:
# 토큰화
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
# tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower = False)

def predict_and_keword(predict_sentence):
  new_data =[[predict_sentence, '0']]
  # print(new_data)

  new_dataset = BERTDataset(new_data, 0, 1, tokenizer, vocab ,max_length, True, False)
  new_dataloader = torch.utils.data.DataLoader(new_dataset, batch_size = batch_size, num_workers =5)

  model.eval()
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(new_dataloader)):
    token_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)
    valid_length = valid_length
    label = label.long().to(device)
    out = model(token_ids, valid_length, segment_ids)

    test_eval =[]
    for logit in out:
      test_eval.append(reverse[np.argmax(logit.detach().cpu().numpy())])

    # 키워드 추출: 명사, 동사, 형용사 추출하기
    keyword = [x for (x, y) in okt.pos(predict_sentence) if y in ['Verb', 'Noun', 'Adjective']]
    print('\nOKT 명사, 동사, 형용사 추출 :', keyword)
    test_eval += keyword

    return test_eval

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [69]:
predict_and_keword('어제 헤어졌어 ㅜ')

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


  0%|          | 0/1 [00:00<?, ?it/s]


OKT 명사, 동사, 형용사 추출 : ['어제', '헤어졌어']


['슬픔', '어제', '헤어졌어']

In [70]:
predict_and_keword('진짜 짜증나')

  0%|          | 0/1 [00:00<?, ?it/s]


OKT 명사, 동사, 형용사 추출 : ['진짜', '짜증나']


['분노', '진짜', '짜증나']

In [71]:
predict_and_keword('너무 무서워')

  0%|          | 0/1 [00:00<?, ?it/s]


OKT 명사, 동사, 형용사 추출 : ['무서워']


['공포', '무서워']

## 성능 올리기
### 1. 중립 데이터 제거해서 테스트
### 2. 파라메터 수정해서 테스트
### 3. 모델 구조 수정?

## 1. 중립 데이터 제거

In [167]:
df_data_neut = df_data[df_data['Emotion'] == '중립'] # 확인

In [168]:
df_data_drop = df_data[df_data['Emotion'] != '중립']
df_data_drop # 중복 데이터 제거한 데이터

,Sentence,Emotion
0,언니 동생으로 부르는게 맞는 일인가요..??,공포
1,그냥 내 느낌일뿐겠지?,공포
2,아직너무초기라서 그런거죠?,공포
3,유치원버스 사고 낫다던데,공포
4,근데 원래이런거맞나요,공포
...,...,...
94132,뭐? 다시 한 번 말해봐.,분노
94152,어? 정말요?,놀람
94154,"혹시, 다들 은행 계좌없는 거예요?",놀람
94169,자네는 대체 뭘 하러 왔나! 젖은 생쥐 꼴이 된 나를 보면서 비웃으러 왔나?,분노


In [206]:
len(df_data), len(df_data_neut)+len(df_data_drop) # 확인

45578

In [170]:
df_data = df_data_drop

In [171]:
target_classes ={
    '공포': 0,
    '놀람': 1,
    '분노': 2,
    '슬픔': 3,
    # '중립': 4,
    '행복': 4,
    '혐오': 5
  }